# Data splits

This notebook focuses on building the data splits to train the model, namely: train, validation and test.

*Constraint: every splits must contain unique subjects, namely different MR session for a subject x, needs to be contained in only ONE split*.

In [2]:
import pandas as pd
import os
import sys

# Append the path to access custom modules
if '../' not in sys.path:
    sys.path.append('../')

import src.utils.data_splitter as ds
import src.utils.dataset_helper as db
import importlib

In [3]:
importlib.reload(ds);
importlib.reload(db);

In [4]:
df = pd.read_csv('../data/csv/final_dataset.csv')
df = df.drop(columns='Unnamed: 0')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2475 entries, 0 to 2474
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Subject                       2475 non-null   object 
 1   MR_session                    2475 non-null   object 
 2   TOTAL_HIPPOCAMPUS_VOLUME      2475 non-null   float64
 3   Left-Hippocampus_volume       2475 non-null   float64
 4   lh_parahippocampal_thickness  2475 non-null   float64
 5   lh_parahippocampal_volume     2475 non-null   int64  
 6   rh_parahippocampal_volume     2475 non-null   int64  
 7   rh_parahippocampal_thickness  2475 non-null   float64
 8   Right-Hippocampus_volume      2475 non-null   float64
 9   label                         2475 non-null   object 
 10  CDR                           2475 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 212.8+ KB


In [6]:
helper = db.DatasetHelper

## Undersample majority class

- Class distribution before resample

In [7]:
df['label'].value_counts()

label
Cognitevely-normal    1891
MCI                    466
Demented               118
Name: count, dtype: int64

In [8]:
df = df.drop(index=df[df['Subject'] == 'OAS31168'].index)

- Get the undersampled dataset that will be the basis dataset from now on

In [9]:
df = df.loc[helper.undersample_majority_class(df['label'], 800)]

- Class distribution after resample

In [10]:
df['label'].value_counts()

label
Cognitevely-normal    800
MCI                   466
Demented              118
Name: count, dtype: int64

- Create the csv file

In [11]:
df.to_csv('../data/csv/final_dataset_undersampled.csv', index=False)

- At this point we can create the new folder with the freesurfers whose MR label is in the resampled df

In [12]:
helper.move_folders(
    subject_experiments=df['MR_session'].to_list(), 
    base_dir=os.path.join('..', 'data', 'freesurfers'),
    dst_dir=os.path.join('..', 'data', 'freesurfers_undersampled')
)

Destination directory is not empty!


## Define the test set

In [13]:
df = pd.read_csv('../data/csv/final_dataset_undersampled.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384 entries, 0 to 1383
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Subject                       1384 non-null   object 
 1   MR_session                    1384 non-null   object 
 2   TOTAL_HIPPOCAMPUS_VOLUME      1384 non-null   float64
 3   Left-Hippocampus_volume       1384 non-null   float64
 4   lh_parahippocampal_thickness  1384 non-null   float64
 5   lh_parahippocampal_volume     1384 non-null   int64  
 6   rh_parahippocampal_volume     1384 non-null   int64  
 7   rh_parahippocampal_thickness  1384 non-null   float64
 8   Right-Hippocampus_volume      1384 non-null   float64
 9   label                         1384 non-null   object 
 10  CDR                           1384 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 119.1+ KB


- Instantiate the data splitter

In [15]:
splitter = ds.DataSplitterTrainTest(df['MR_session'], df['label'])

- Retrieve the the train and test instances

In [16]:
train_idx, test_idx = splitter.get_train_test_set_idx()

- Check train label distribution

In [17]:
df.loc[train_idx]['label'].value_counts()

label
Cognitevely-normal    667
MCI                   395
Demented               88
Name: count, dtype: int64

- Check test label distribution

In [18]:
df.loc[test_idx]['label'].value_counts()

label
Cognitevely-normal    133
MCI                    71
Demented               30
Name: count, dtype: int64

- Create a csv file related to test instances

In [19]:
df.loc[test_idx].to_csv('../data/csv/test.csv')

- Test volumetric images can now be moved in the dedicated folder

In [20]:
helper.move_folders(
    subject_experiments=df.loc[test_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'test_freesurfers')
)

Destination directory is not empty!


## Define validation set

We can procede like before to define the validation set, to evaluate the model during training

In [21]:
# Work on a copy of the training set only
X_train = df.loc[train_idx].copy(deep=True) 

- Split the data

In [22]:
splitter = ds.DataSplitterTrainTest(X_train['MR_session'], X_train['label'])
train_idx, val_idx = splitter.get_train_test_set_idx()

In [23]:
X_train.loc[train_idx, 'label'].value_counts()

label
Cognitevely-normal    567
MCI                   342
Demented               69
Name: count, dtype: int64

In [24]:
X_train.loc[val_idx, 'label'].value_counts()

label
Cognitevely-normal    100
MCI                    53
Demented               19
Name: count, dtype: int64

- Move train set images to a separate folder

In [24]:
helper.move_folders(
    subject_experiments=X_train.loc[train_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'train_freesurfers')
)

- Move the validation set images to a separate folder

In [25]:
helper.move_folders(
    subject_experiments=X_train.loc[val_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'validation_freesurfers')
)

- Create clean (with no augmentated instances) csv file related to train and validation set

In [26]:
X_train.loc[train_idx].to_csv('../data/csv/train_clean.csv')

## Add augmentation labels to train set

- This section will add to the train dataset (train + validation) the augmented instances to improve class balance
- Let's start by augmenting MCIs

In [25]:
mci_aug_train = helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'MCI'].sample(n=20, random_state=42),
    id_col_name='MR_session',
    suffix_list=['flip', 'rot30', 'rot60', 'rot90']
)

- Augment demented label

In [26]:
dem_aug_train = helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Demented'],
    id_col_name='MR_session',
    suffix_list=['flip', 'rot30', 'rot60', 'rot90']
)

- Get the final training dataframe

In [27]:
X_train_final = pd.concat(objs=[X_train.loc[train_idx]] + mci_aug_train + dem_aug_train, ignore_index=True)

- This is the final distribution of the labels on the training set

In [30]:
X_train_final['label'].value_counts()

label
Cognitevely-normal    567
MCI                   422
Demented              345
Name: count, dtype: int64

- This is the final distribution of labels on the validation set

In [31]:
X_train.loc[val_idx, 'label'].value_counts()

label
Cognitevely-normal    100
MCI                    53
Demented               19
Name: count, dtype: int64

- Build a new column to indicate the augmentation operation to perform for a specific instance

In [32]:
X_train_final['augmentation'] = helper.extract_augmentation_column(X_train_final['MR_session'])

In [33]:
X_train_final['MR_session_original'] = X_train_final['MR_session_original'].fillna(value='normal')

- Save the augmented dataset to csv

In [34]:
X_train_final.to_csv('../data/csv/train.csv')
X_train.loc[val_idx, :].to_csv('../data/csv/validation.csv')

In [35]:
idx = helper.undersample_majority_class(X_train_final['label'], 467)

In [36]:
X_train_final = X_train_final.loc[idx, :]

In [37]:
X_train_final.to_csv('../data/csv/train.csv')

In [38]:
X_train_final[X_train_final['augmentation'] == 'normal'].to_csv('../data/csv/train_clean.csv')

In [41]:
X_train_final['label'].value_counts()

label
Cognitevely-normal    467
MCI                   422
Demented              345
Name: count, dtype: int64